# Libraries

In [ ]:
!pip install catboost

In [ ]:
from catboost import CatBoostRegressor
import pandas as pd
from sklearn.model_selection import KFold
import os

# Data

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# Prediction

In [ ]:
min_max_catboost = '/content/drive/MyDrive/payroll/stacked_models/min_max_catboost/'
min_max_catboost_7575 = '/content/drive/MyDrive/payroll/stacked_models/min_max_catboost_7575/'
min_max_catboost_69 = '/content/drive/MyDrive/payroll/stacked_models/min_max_catboost_69/'
min_max_catboost_23 = '/content/drive/MyDrive/payroll/stacked_models/min_max_catboost_23/'

models = os.listdir(min_max_catboost)
models_7575 = os.listdir(min_max_catboost_7575)
models_69 = os.listdir(min_max_catboost_69)
models_23 = os.listdir(min_max_catboost_23)

In [ ]:
def get_preds(all_models, base_stack_path):
    all_preds = []
    for model in tqdm(all_models):
        estimator = CatBoostRegressor(
                          iterations=15000,
                          learning_rate=0.00065,
                          depth=11,
                          l2_leaf_reg=4,
                          # model_size_reg=None,
                          loss_function='MAE',
                          eval_metric="MAE",
                          verbose=1000,
                          random_strength=2,
                          # random_state=42,
                          task_type="GPU",
                          # objective="MSE",
            )  
        estimator.load_model(base_stack_path + model)

        preds = np.exp(estimator.predict(test)) - 1
        
        all_preds.append(preds)

    prediction = all_preds[0]
    for i in range(1, 6):
        prediction += all_preds[i]
        
    prediction = prediction / 6
    prediction

    return prediction


In [ ]:
min_max_catboost_preds = get_preds(models, min_max_catboost)
min_max_catboost_7575_preds = get_preds(models_7575, min_max_catboost_7575)
min_max_catboost_69_preds = get_preds(models_69, min_max_catboost_69)
min_max_catboost_23_preds = get_preds(models_23, min_max_catboost_23)

In [ ]:
submission = pd.read_csv(f'/content/drive/MyDrive/payroll/data/sample_submission.csv', index_col=0)

In [ ]:
submission["mean_salary"] = (min_max_catboost_preds + min_max_catboost_7575_preds + min_max_catboost_69_preds + min_max_catboost_23_preds) / 4
submission

In [ ]:
submission.to_csv("final_submission.csv")